In [1]:
import pandas as pd
import numpy as np
import scipy.sparse
from xgboost import XGBClassifier

In [2]:
import xgboost

In [3]:

import sklearn
import matplotlib.pyplot as plt


In [4]:
from catboost import CatBoostClassifier

In [57]:
train_df=pd.read_csv('train_sessions.csv')
test_df=pd.read_csv('test_sessions.csv')

In [6]:
train_df.drop('session_id',axis=1,inplace=True)

In [7]:
test_df.drop('session_id',axis=1,inplace=True)

In [8]:
y=train_df.target

In [9]:
train_df.drop('target',axis=1,inplace=True)

In [10]:
sh=train_df.shape[0]

In [11]:
full_df=pd.concat([train_df,test_df])

In [12]:
full_df.reset_index(inplace=True)

In [13]:
full_df.drop('index',axis=1,inplace=True)

In [14]:
full_df=full_df.fillna(0)

In [15]:
sites=full_df.filter(like='site')

In [16]:
from scipy.sparse import csr_matrix, hstack

In [17]:
sites_flatten=sites.values.flatten()

In [18]:
sites_csr=csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]

In [19]:
sites_csr

<336358x48371 sparse matrix of type '<class 'numpy.int64'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [20]:
full_df['data']=pd.DatetimeIndex(full_df.time1).year*100+pd.DatetimeIndex(full_df.time1).month

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
scaler=StandardScaler()

In [23]:
full_df.data=scaler.fit_transform(np.array(full_df.data).reshape(-1,1))

In [24]:
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,data
0,718,2014-02-20 10:02:45,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.476232
1,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0.476232
2,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,-1.800775
3,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0.501532
4,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0.476232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,0.0,0,0.0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.678632
336354,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,2014-05-26 14:17:19,...,2014-05-26 14:17:19,302.0,2014-05-26 14:17:19,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19,0.552132
336355,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,2014-05-02 11:22:03,...,2014-05-02 11:22:03,30.0,2014-05-02 11:22:03,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04,0.552132
336356,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:36,...,2014-05-03 10:05:37,23.0,2014-05-03 10:05:37,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38,0.552132


In [25]:
full_df['start_month']=pd.DatetimeIndex(full_df.time1).month

In [26]:
full_df['start_hour']=pd.DatetimeIndex(full_df.time1).hour
full_df['morning']=(pd.DatetimeIndex(full_df.time1).hour<=11)

In [27]:
full_df.morning=full_df.morning.map({True:1,False:0})

In [28]:
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site8,time8,site9,time9,site10,time10,data,start_month,start_hour,morning
0,718,2014-02-20 10:02:45,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.476232,2,10,1
1,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0.476232,2,11,1
2,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,-1.800775,12,16,0
3,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0.501532,3,10,1
4,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0.476232,2,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.678632,10,18,0
336354,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,2014-05-26 14:17:19,...,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19,0.552132,5,14,0
336355,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,2014-05-02 11:22:03,...,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04,0.552132,5,11,1
336356,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:36,...,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38,0.552132,5,10,1


In [29]:
X_train=hstack([sites_csr[:sh],full_df.iloc[:sh,[-1,-2,-3,-4]]])

In [30]:
X_test=hstack([sites_csr[sh:],full_df.iloc[sh:,[-1,-2,-3,-4]]])

In [31]:
y_train=y[:sh]

In [32]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
253556    0
253557    0
253558    0
253559    0
253560    0
Name: target, Length: 253561, dtype: int64

In [33]:
X_train

<253561x48375 sparse matrix of type '<class 'numpy.float64'>'
	with 3295488 stored elements in COOrdinate format>

In [34]:
xgb1=XGBClassifier(random_state=17,use_label_encoder=False, n_jobs=-1)

In [35]:
from sklearn.linear_model import LogisticRegression as glm

In [47]:
fit=glm(max_iter=10000)

In [36]:
sites=pd.concat([sites,full_df[['data','start_hour','start_month','morning']]],axis=1)

In [38]:
a=sites_T>0

In [37]:
sites_T=sites.filter(like='site').T

In [42]:
sites['n_sites']=a

In [43]:
sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,data,start_hour,start_month,morning,n_sites
0,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476232,10,2,1,1
1,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0,0.476232,11,2,1,10
2,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0,-1.800775,16,12,0,10
3,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,0.501532,10,3,1,10
4,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0,0.476232,10,2,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,1039.0,676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678632,18,10,0,3
336354,300,302.0,302.0,300.0,300.0,1222.0,302.0,1218.0,1221.0,1216.0,0.552132,14,5,0,10
336355,29,33.0,35.0,22.0,37.0,6779.0,30.0,21.0,23.0,6780.0,0.552132,11,5,1,10
336356,5828,23.0,21.0,804.0,21.0,3350.0,23.0,894.0,21.0,961.0,0.552132,10,5,1,10


In [44]:
sites['week_day']=pd.DatetimeIndex(full_df.time1).dayofweek

In [45]:
sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,data,start_hour,start_month,morning,n_sites,week_day
0,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476232,10,2,1,1,3
1,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0,0.476232,11,2,1,10,5
2,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0,-1.800775,16,12,0,10,0
3,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,0.501532,10,3,1,10,4
4,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0,0.476232,10,2,1,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,1039.0,676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678632,18,10,0,3,3
336354,300,302.0,302.0,300.0,300.0,1222.0,302.0,1218.0,1221.0,1216.0,0.552132,14,5,0,10,0
336355,29,33.0,35.0,22.0,37.0,6779.0,30.0,21.0,23.0,6780.0,0.552132,11,5,1,10,4
336356,5828,23.0,21.0,804.0,21.0,3350.0,23.0,894.0,21.0,961.0,0.552132,10,5,1,10,5


In [46]:
dict_s=pd.read_pickle('site_dic.pkl')

In [49]:
dict_s

{'www.abmecatronique.com': 25075,
 'groups.live.com': 13997,
 'majeureliguefootball.wordpress.com': 42436,
 'cdt46.media.tourinsoft.eu': 30911,
 'www.hdwallpapers.eu': 8104,
 'img378.imageshack.us': 37491,
 'ecologie.nature.free.fr': 5462,
 'www.ibcn.intec.ugent.be': 35425,
 'kissanime.com': 30151,
 'www.carolineconduiteformation.com': 38268,
 'images.mystockphoto.com': 43641,
 'journalph.csphares.qc.ca': 36959,
 'www.uqo.ca': 40935,
 'd8d94e0wul1nb.cloudfront.net': 12346,
 'openapi.elong.com': 31023,
 'flamenco-o.blogspot.com': 31114,
 'www.pages-annuaire.net': 28165,
 'smart2000.pagesperso-orange.fr': 31877,
 'fast.forbes.com': 31070,
 'i1-js-14-3-01-10077-536503633-i.init.cedexis-radar.net': 12938,
 'i1-js-14-3-01-11074-716595896-i.init.cedexis-radar.net': 42002,
 'www.pacajob.com': 5671,
 'mathaa.epfl.ch': 32074,
 'cbv.sfr.bench.cedexis.com': 7104,
 'fbcdn-sphotos-b-a.akamaihd.net': 1939,
 'www.mystere-tv.com': 6708,
 'www.mon-ip.fr': 43589,
 'www.aqua-passion.com': 45241,
 'reunio

In [50]:
from catboost import CatBoostClassifier

In [51]:
cat=CatBoostClassifier()

In [54]:
cat.fit(sites[:sh],y)

Learning rate set to 0.109516
0:	learn: 0.4655249	total: 81ms	remaining: 1m 20s
1:	learn: 0.2938876	total: 100ms	remaining: 49.9s
2:	learn: 0.1882118	total: 121ms	remaining: 40.2s
3:	learn: 0.1338445	total: 141ms	remaining: 35.2s
4:	learn: 0.0939972	total: 162ms	remaining: 32.3s
5:	learn: 0.0727483	total: 183ms	remaining: 30.3s
6:	learn: 0.0617573	total: 205ms	remaining: 29.1s
7:	learn: 0.0541923	total: 228ms	remaining: 28.2s
8:	learn: 0.0487030	total: 249ms	remaining: 27.4s
9:	learn: 0.0442261	total: 270ms	remaining: 26.7s
10:	learn: 0.0410799	total: 291ms	remaining: 26.2s
11:	learn: 0.0378676	total: 313ms	remaining: 25.8s
12:	learn: 0.0360028	total: 334ms	remaining: 25.3s
13:	learn: 0.0346169	total: 355ms	remaining: 25s
14:	learn: 0.0329419	total: 376ms	remaining: 24.7s
15:	learn: 0.0319185	total: 397ms	remaining: 24.4s
16:	learn: 0.0313653	total: 417ms	remaining: 24.1s
17:	learn: 0.0309560	total: 441ms	remaining: 24.1s
18:	learn: 0.0303240	total: 464ms	remaining: 23.9s
19:	learn: 0.

162:	learn: 0.0168159	total: 3.68s	remaining: 18.9s
163:	learn: 0.0167791	total: 3.7s	remaining: 18.9s
164:	learn: 0.0167475	total: 3.72s	remaining: 18.8s
165:	learn: 0.0167161	total: 3.75s	remaining: 18.8s
166:	learn: 0.0166663	total: 3.77s	remaining: 18.8s
167:	learn: 0.0166168	total: 3.79s	remaining: 18.8s
168:	learn: 0.0166023	total: 3.81s	remaining: 18.7s
169:	learn: 0.0165537	total: 3.83s	remaining: 18.7s
170:	learn: 0.0165133	total: 3.85s	remaining: 18.7s
171:	learn: 0.0164955	total: 3.87s	remaining: 18.6s
172:	learn: 0.0164621	total: 3.89s	remaining: 18.6s
173:	learn: 0.0164303	total: 3.92s	remaining: 18.6s
174:	learn: 0.0163719	total: 3.94s	remaining: 18.6s
175:	learn: 0.0163455	total: 3.96s	remaining: 18.5s
176:	learn: 0.0163156	total: 3.98s	remaining: 18.5s
177:	learn: 0.0162673	total: 4s	remaining: 18.5s
178:	learn: 0.0162491	total: 4.02s	remaining: 18.5s
179:	learn: 0.0162037	total: 4.04s	remaining: 18.4s
180:	learn: 0.0161537	total: 4.07s	remaining: 18.4s
181:	learn: 0.01

329:	learn: 0.0124146	total: 7.38s	remaining: 15s
330:	learn: 0.0124007	total: 7.4s	remaining: 15s
331:	learn: 0.0123781	total: 7.42s	remaining: 14.9s
332:	learn: 0.0123578	total: 7.44s	remaining: 14.9s
333:	learn: 0.0123283	total: 7.46s	remaining: 14.9s
334:	learn: 0.0123133	total: 7.49s	remaining: 14.9s
335:	learn: 0.0123019	total: 7.51s	remaining: 14.8s
336:	learn: 0.0122723	total: 7.53s	remaining: 14.8s
337:	learn: 0.0122489	total: 7.55s	remaining: 14.8s
338:	learn: 0.0122300	total: 7.58s	remaining: 14.8s
339:	learn: 0.0122171	total: 7.6s	remaining: 14.8s
340:	learn: 0.0121964	total: 7.62s	remaining: 14.7s
341:	learn: 0.0121763	total: 7.65s	remaining: 14.7s
342:	learn: 0.0121590	total: 7.67s	remaining: 14.7s
343:	learn: 0.0121459	total: 7.69s	remaining: 14.7s
344:	learn: 0.0121260	total: 7.71s	remaining: 14.6s
345:	learn: 0.0121208	total: 7.74s	remaining: 14.6s
346:	learn: 0.0121118	total: 7.76s	remaining: 14.6s
347:	learn: 0.0120881	total: 7.78s	remaining: 14.6s
348:	learn: 0.0120

495:	learn: 0.0098835	total: 11s	remaining: 11.2s
496:	learn: 0.0098660	total: 11.1s	remaining: 11.2s
497:	learn: 0.0098530	total: 11.1s	remaining: 11.2s
498:	learn: 0.0098446	total: 11.1s	remaining: 11.1s
499:	learn: 0.0098330	total: 11.1s	remaining: 11.1s
500:	learn: 0.0098189	total: 11.1s	remaining: 11.1s
501:	learn: 0.0098105	total: 11.2s	remaining: 11.1s
502:	learn: 0.0098014	total: 11.2s	remaining: 11.1s
503:	learn: 0.0097779	total: 11.2s	remaining: 11s
504:	learn: 0.0097632	total: 11.2s	remaining: 11s
505:	learn: 0.0097538	total: 11.3s	remaining: 11s
506:	learn: 0.0097412	total: 11.3s	remaining: 11s
507:	learn: 0.0097293	total: 11.3s	remaining: 10.9s
508:	learn: 0.0097108	total: 11.3s	remaining: 10.9s
509:	learn: 0.0096920	total: 11.3s	remaining: 10.9s
510:	learn: 0.0096795	total: 11.4s	remaining: 10.9s
511:	learn: 0.0096697	total: 11.4s	remaining: 10.9s
512:	learn: 0.0096558	total: 11.4s	remaining: 10.8s
513:	learn: 0.0096460	total: 11.4s	remaining: 10.8s
514:	learn: 0.0096400	

662:	learn: 0.0081167	total: 14.7s	remaining: 7.47s
663:	learn: 0.0081090	total: 14.7s	remaining: 7.45s
664:	learn: 0.0080989	total: 14.7s	remaining: 7.42s
665:	learn: 0.0080911	total: 14.8s	remaining: 7.4s
666:	learn: 0.0080862	total: 14.8s	remaining: 7.38s
667:	learn: 0.0080781	total: 14.8s	remaining: 7.36s
668:	learn: 0.0080700	total: 14.8s	remaining: 7.33s
669:	learn: 0.0080565	total: 14.8s	remaining: 7.31s
670:	learn: 0.0080515	total: 14.9s	remaining: 7.29s
671:	learn: 0.0080426	total: 14.9s	remaining: 7.27s
672:	learn: 0.0080364	total: 14.9s	remaining: 7.24s
673:	learn: 0.0080268	total: 14.9s	remaining: 7.22s
674:	learn: 0.0080233	total: 15s	remaining: 7.2s
675:	learn: 0.0080121	total: 15s	remaining: 7.18s
676:	learn: 0.0080050	total: 15s	remaining: 7.15s
677:	learn: 0.0080008	total: 15s	remaining: 7.13s
678:	learn: 0.0079950	total: 15s	remaining: 7.11s
679:	learn: 0.0079914	total: 15.1s	remaining: 7.08s
680:	learn: 0.0079846	total: 15.1s	remaining: 7.06s
681:	learn: 0.0079830	to

822:	learn: 0.0069081	total: 18.1s	remaining: 3.9s
823:	learn: 0.0068980	total: 18.2s	remaining: 3.88s
824:	learn: 0.0068954	total: 18.2s	remaining: 3.85s
825:	learn: 0.0068917	total: 18.2s	remaining: 3.83s
826:	learn: 0.0068874	total: 18.2s	remaining: 3.81s
827:	learn: 0.0068819	total: 18.2s	remaining: 3.79s
828:	learn: 0.0068759	total: 18.3s	remaining: 3.77s
829:	learn: 0.0068639	total: 18.3s	remaining: 3.74s
830:	learn: 0.0068610	total: 18.3s	remaining: 3.72s
831:	learn: 0.0068525	total: 18.3s	remaining: 3.7s
832:	learn: 0.0068415	total: 18.3s	remaining: 3.68s
833:	learn: 0.0068317	total: 18.4s	remaining: 3.65s
834:	learn: 0.0068234	total: 18.4s	remaining: 3.63s
835:	learn: 0.0068182	total: 18.4s	remaining: 3.61s
836:	learn: 0.0068081	total: 18.4s	remaining: 3.59s
837:	learn: 0.0067963	total: 18.4s	remaining: 3.57s
838:	learn: 0.0067871	total: 18.5s	remaining: 3.54s
839:	learn: 0.0067806	total: 18.5s	remaining: 3.52s
840:	learn: 0.0067704	total: 18.5s	remaining: 3.5s
841:	learn: 0.0

982:	learn: 0.0059709	total: 21.6s	remaining: 373ms
983:	learn: 0.0059686	total: 21.6s	remaining: 351ms
984:	learn: 0.0059669	total: 21.6s	remaining: 329ms
985:	learn: 0.0059593	total: 21.6s	remaining: 307ms
986:	learn: 0.0059546	total: 21.7s	remaining: 285ms
987:	learn: 0.0059519	total: 21.7s	remaining: 263ms
988:	learn: 0.0059474	total: 21.7s	remaining: 242ms
989:	learn: 0.0059418	total: 21.7s	remaining: 220ms
990:	learn: 0.0059363	total: 21.8s	remaining: 198ms
991:	learn: 0.0059299	total: 21.8s	remaining: 176ms
992:	learn: 0.0059244	total: 21.8s	remaining: 154ms
993:	learn: 0.0059191	total: 21.8s	remaining: 132ms
994:	learn: 0.0059114	total: 21.8s	remaining: 110ms
995:	learn: 0.0059028	total: 21.9s	remaining: 87.8ms
996:	learn: 0.0058998	total: 21.9s	remaining: 65.9ms
997:	learn: 0.0058968	total: 21.9s	remaining: 43.9ms
998:	learn: 0.0058950	total: 21.9s	remaining: 22ms
999:	learn: 0.0058897	total: 22s	remaining: 0us


In [55]:
pred=cat.predict(sites[sh:])

In [73]:
def save(pred):
    pd.DataFrame({'session_id':test_df.session_id,'target':pred}).to_csv('alice.csv',index=False)

In [58]:
test_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,...,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,...,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,...,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,...,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,...,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82792,82793,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82793,82794,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,...,1222.0,2014-05-26 14:17:19,302.0,2014-05-26 14:17:19,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19
82794,82795,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,...,6779.0,2014-05-02 11:22:03,30.0,2014-05-02 11:22:03,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04
82795,82796,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,...,3350.0,2014-05-03 10:05:37,23.0,2014-05-03 10:05:37,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38


In [61]:
from sklearn.preprocessing import OneHotEncoder

In [62]:
enc=OneHotEncoder()

In [87]:
sites_sp=enc.fit_transform(sites.drop(['n_sites','data'],axis=1))

In [88]:
from scipy.sparse import hstack, csr_matrix

In [90]:
sites_sp=hstack([sites_sp,sites[['n_sites','data']]])

In [91]:
sites_sp

<336358x183240 sparse matrix of type '<class 'numpy.float64'>'
	with 5718086 stored elements in COOrdinate format>

In [92]:
from xgboost import XGBClassifier

In [93]:
xgb=XGBClassifier(use_label_encoder=False,eval_metric='aucpr')

In [94]:
sites_sp=csr_matrix(sites_sp)

In [95]:
xgb.fit(sites_sp[:sh],y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='aucpr',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [96]:
save(xgb.predict_proba(sites_sp[sh:])[:,1])

In [82]:
xgb.predict_proba(sites_sp[sh:])[]

array([9.9986619e-01, 1.3379061e-04], dtype=float32)

In [85]:
sites

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,data,start_hour,start_month,morning,n_sites,week_day
0,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.476232,10,2,1,1,3
1,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0,0.476232,11,2,1,10,5
2,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0,-1.800775,16,12,0,10,0
3,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,0.501532,10,3,1,10,4
4,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0,0.476232,10,2,1,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,1039.0,676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678632,18,10,0,3,3
336354,300,302.0,302.0,300.0,300.0,1222.0,302.0,1218.0,1221.0,1216.0,0.552132,14,5,0,10,0
336355,29,33.0,35.0,22.0,37.0,6779.0,30.0,21.0,23.0,6780.0,0.552132,11,5,1,10,4
336356,5828,23.0,21.0,804.0,21.0,3350.0,23.0,894.0,21.0,961.0,0.552132,10,5,1,10,5


In [86]:
full_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site8,time8,site9,time9,site10,time10,data,start_month,start_hour,morning
0,718,2014-02-20 10:02:45,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.476232,2,10,1
1,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0.476232,2,11,1
2,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,-1.800775,12,16,0
3,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0.501532,3,10,1
4,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0.476232,2,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336353,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.678632,10,18,0
336354,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,2014-05-26 14:17:19,...,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19,0.552132,5,14,0
336355,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,2014-05-02 11:22:03,...,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04,0.552132,5,11,1
336356,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:36,...,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38,0.552132,5,10,1


In [100]:
cat=CatBoostClassifier(eval_metric='AUC')

In [101]:
cat.fit(sites[:sh],y)

Learning rate set to 0.109516
0:	total: 21.2ms	remaining: 21.2s
1:	total: 43.2ms	remaining: 21.6s
2:	total: 65.4ms	remaining: 21.7s
3:	total: 86.9ms	remaining: 21.6s
4:	total: 108ms	remaining: 21.5s
5:	total: 130ms	remaining: 21.6s
6:	total: 153ms	remaining: 21.7s
7:	total: 175ms	remaining: 21.7s
8:	total: 197ms	remaining: 21.6s
9:	total: 221ms	remaining: 21.9s
10:	total: 243ms	remaining: 21.9s
11:	total: 267ms	remaining: 21.9s
12:	total: 288ms	remaining: 21.8s
13:	total: 309ms	remaining: 21.8s
14:	total: 331ms	remaining: 21.7s
15:	total: 351ms	remaining: 21.6s
16:	total: 371ms	remaining: 21.4s
17:	total: 392ms	remaining: 21.4s
18:	total: 413ms	remaining: 21.3s
19:	total: 435ms	remaining: 21.3s
20:	total: 457ms	remaining: 21.3s
21:	total: 479ms	remaining: 21.3s
22:	total: 500ms	remaining: 21.3s
23:	total: 522ms	remaining: 21.2s
24:	total: 543ms	remaining: 21.2s
25:	total: 564ms	remaining: 21.1s
26:	total: 585ms	remaining: 21.1s
27:	total: 606ms	remaining: 21s
28:	total: 627ms	remaining

241:	total: 5.54s	remaining: 17.3s
242:	total: 5.56s	remaining: 17.3s
243:	total: 5.58s	remaining: 17.3s
244:	total: 5.6s	remaining: 17.3s
245:	total: 5.62s	remaining: 17.2s
246:	total: 5.64s	remaining: 17.2s
247:	total: 5.67s	remaining: 17.2s
248:	total: 5.69s	remaining: 17.2s
249:	total: 5.71s	remaining: 17.1s
250:	total: 5.73s	remaining: 17.1s
251:	total: 5.75s	remaining: 17.1s
252:	total: 5.78s	remaining: 17.1s
253:	total: 5.8s	remaining: 17s
254:	total: 5.82s	remaining: 17s
255:	total: 5.84s	remaining: 17s
256:	total: 5.86s	remaining: 16.9s
257:	total: 5.88s	remaining: 16.9s
258:	total: 5.9s	remaining: 16.9s
259:	total: 5.92s	remaining: 16.9s
260:	total: 5.94s	remaining: 16.8s
261:	total: 5.96s	remaining: 16.8s
262:	total: 5.99s	remaining: 16.8s
263:	total: 6.01s	remaining: 16.8s
264:	total: 6.03s	remaining: 16.7s
265:	total: 6.05s	remaining: 16.7s
266:	total: 6.08s	remaining: 16.7s
267:	total: 6.09s	remaining: 16.6s
268:	total: 6.12s	remaining: 16.6s
269:	total: 6.14s	remaining: 

483:	total: 10.9s	remaining: 11.6s
484:	total: 10.9s	remaining: 11.6s
485:	total: 10.9s	remaining: 11.6s
486:	total: 10.9s	remaining: 11.5s
487:	total: 11s	remaining: 11.5s
488:	total: 11s	remaining: 11.5s
489:	total: 11s	remaining: 11.5s
490:	total: 11s	remaining: 11.4s
491:	total: 11.1s	remaining: 11.4s
492:	total: 11.1s	remaining: 11.4s
493:	total: 11.1s	remaining: 11.4s
494:	total: 11.1s	remaining: 11.4s
495:	total: 11.1s	remaining: 11.3s
496:	total: 11.2s	remaining: 11.3s
497:	total: 11.2s	remaining: 11.3s
498:	total: 11.2s	remaining: 11.3s
499:	total: 11.2s	remaining: 11.2s
500:	total: 11.3s	remaining: 11.2s
501:	total: 11.3s	remaining: 11.2s
502:	total: 11.3s	remaining: 11.2s
503:	total: 11.3s	remaining: 11.2s
504:	total: 11.4s	remaining: 11.1s
505:	total: 11.4s	remaining: 11.1s
506:	total: 11.4s	remaining: 11.1s
507:	total: 11.4s	remaining: 11.1s
508:	total: 11.4s	remaining: 11s
509:	total: 11.5s	remaining: 11s
510:	total: 11.5s	remaining: 11s
511:	total: 11.5s	remaining: 11s
5

723:	total: 16.3s	remaining: 6.2s
724:	total: 16.3s	remaining: 6.18s
725:	total: 16.3s	remaining: 6.15s
726:	total: 16.3s	remaining: 6.13s
727:	total: 16.4s	remaining: 6.11s
728:	total: 16.4s	remaining: 6.09s
729:	total: 16.4s	remaining: 6.06s
730:	total: 16.4s	remaining: 6.04s
731:	total: 16.4s	remaining: 6.02s
732:	total: 16.5s	remaining: 5.99s
733:	total: 16.5s	remaining: 5.97s
734:	total: 16.5s	remaining: 5.95s
735:	total: 16.5s	remaining: 5.93s
736:	total: 16.5s	remaining: 5.9s
737:	total: 16.6s	remaining: 5.88s
738:	total: 16.6s	remaining: 5.86s
739:	total: 16.6s	remaining: 5.84s
740:	total: 16.6s	remaining: 5.81s
741:	total: 16.7s	remaining: 5.79s
742:	total: 16.7s	remaining: 5.77s
743:	total: 16.7s	remaining: 5.75s
744:	total: 16.7s	remaining: 5.72s
745:	total: 16.7s	remaining: 5.7s
746:	total: 16.8s	remaining: 5.68s
747:	total: 16.8s	remaining: 5.65s
748:	total: 16.8s	remaining: 5.63s
749:	total: 16.8s	remaining: 5.61s
750:	total: 16.8s	remaining: 5.58s
751:	total: 16.9s	remai

968:	total: 21.8s	remaining: 697ms
969:	total: 21.8s	remaining: 674ms
970:	total: 21.8s	remaining: 652ms
971:	total: 21.8s	remaining: 629ms
972:	total: 21.9s	remaining: 607ms
973:	total: 21.9s	remaining: 584ms
974:	total: 21.9s	remaining: 562ms
975:	total: 21.9s	remaining: 539ms
976:	total: 22s	remaining: 517ms
977:	total: 22s	remaining: 494ms
978:	total: 22s	remaining: 472ms
979:	total: 22s	remaining: 449ms
980:	total: 22s	remaining: 427ms
981:	total: 22.1s	remaining: 405ms
982:	total: 22.1s	remaining: 382ms
983:	total: 22.1s	remaining: 360ms
984:	total: 22.1s	remaining: 337ms
985:	total: 22.2s	remaining: 315ms
986:	total: 22.2s	remaining: 292ms
987:	total: 22.2s	remaining: 270ms
988:	total: 22.2s	remaining: 247ms
989:	total: 22.3s	remaining: 225ms
990:	total: 22.3s	remaining: 202ms
991:	total: 22.3s	remaining: 180ms
992:	total: 22.3s	remaining: 157ms
993:	total: 22.3s	remaining: 135ms
994:	total: 22.4s	remaining: 112ms
995:	total: 22.4s	remaining: 89.9ms
996:	total: 22.4s	remaining: 

In [103]:
save(cat.predict_proba(sites[sh:])[:,1])

In [104]:
sites_sp

<336358x183240 sparse matrix of type '<class 'numpy.float64'>'
	with 5718086 stored elements in Compressed Sparse Row format>